In [5]:
# # IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATASETS
# # TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# # THEN FEEL FREE TO DELETE CELL.

# import os
# import sys
# from tempfile import NamedTemporaryFile
# from urllib.request import urlopen
# from urllib.parse import unquote
# from urllib.error import HTTPError
# from zipfile import ZipFile

# CHUNK_SIZE = 40960
# DATASET_MAPPING = ':https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F729820%2F1267148%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20230902%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20230902T203321Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D89a2e63a48c344a6b7acab1d8b170cef0eeb843c2fe13eda5e469b4daacb98e0fc60f7447a3415c6702e8b16f5545903e682cc89ac17b0b5c1b9e89f60e69f8beda15e9ce9bb726487e86c657fb19de889253469653fa7a16c1a03cac3b7d0ae3df9e57d6b1577078826420ba1212ec74050921e8aa86be61c0037b7950b164a02ae79837542dfcd9451eb49c13864337c4789f25baacec48c517287de343d2eda6a6e4d6b91dd515ebfac95ba25b16ae1658cc1f784a22bfce7a26ec7423bbcb701b31660278386223440fdc7dd480840609f0b1898657663aedc365179b9d0ecb65df9618f19a8d8303c0417c91d065cdbda8e5dece2c31ab8def35d52e047'
# KAGGLE_INPUT_PATH='/home/kaggle/input'
# KAGGLE_INPUT_SYMLINK='/kaggle'

# os.makedirs(KAGGLE_INPUT_PATH, 777)
# os.symlink(KAGGLE_INPUT_PATH, os.path.join('..', 'input'), target_is_directory=True)
# os.makedirs(KAGGLE_INPUT_SYMLINK)
# os.symlink(KAGGLE_INPUT_PATH, os.path.join(KAGGLE_INPUT_SYMLINK, 'input'), target_is_directory=True)

# for dataset_mapping in DATASET_MAPPING.split(','):
#     directory, download_url_encoded = dataset_mapping.split(':')
#     download_url = unquote(download_url_encoded)
#     destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
#     try:
#         with urlopen(download_url) as zipfileres, NamedTemporaryFile() as tfile:
#             total_length = zipfileres.headers['content-length']
#             print(f'Downloading {directory}, {total_length} bytes zipped')
#             dl = 0
#             data = zipfileres.read(CHUNK_SIZE)
#             while len(data) > 0:
#                 dl += len(data)
#                 tfile.write(data)
#                 done = int(50 * dl / int(total_length))
#                 sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
#                 sys.stdout.flush()
#                 data = zipfileres.read(CHUNK_SIZE)
#             print(f'\nUnzipping {directory}')
#             with ZipFile(tfile) as zfile:
#                 zfile.extractall(destination_path)
#     except HTTPError as e:
#         print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
#         continue
#     except OSError as e:
#         print(f'Failed to load {download_url} to path {destination_path}')
#         continue
# print('Dataset import complete.')


PermissionError: [Errno 13] Permission denied: '/home/kaggle'

In [ ]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import random
from matplotlib.patches import Rectangle
from lxml import etree
%matplotlib inline

### 1. Data Preprocessing

#### 1.1 Image_Label_Preview

In [ ]:
import os
os.listdir('../input/images/images')

In [ ]:
os.listdir('../input/label/label')

#### 1.2 Load Data

In [ ]:
image_path = glob.glob('../input/images/images/*/*.jpg')
len(image_path)

In [ ]:
image_path[:3]

In [ ]:
xmls_path = glob.glob('../input/label/label/*.xml')
len(xmls_path)

In [ ]:
xmls_path[:3]

In [ ]:
#xml_name extraction
xmls_train = [p.split('/')[-1].split('.')[0] for p in xmls_path]
xmls_train[:3]

In [ ]:
#img_name extraction
imgs_train = [img for img in image_path if (img.split('/')[-1].split)('.jpg')[0] in xmls_train]
imgs_train[:3]

In [ ]:
len(imgs_train),len(xmls_path)

In [ ]:
#check the image to label sorts
xmls_path.sort(key=lambda x:x.split('/')[-1].split('.xml')[0])
imgs_train.sort(key=lambda x:x.split('/')[-1].split('.jpg')[0])
xmls_path[:3],imgs_train[:3]

In [ ]:
#labels names
names = [x.split("/")[-2] for x in imgs_train]
names[:3]

In [ ]:
names = pd.DataFrame(names,columns=['Types'])
names

In [ ]:
#onehot for mutiple classes
from sklearn.preprocessing import LabelBinarizer

Class = names['Types'].unique()
Class_dict = dict(zip(Class, range(1,len(Class)+1)))
names['str'] = names['Types'].apply(lambda x: Class_dict[x])
lb = LabelBinarizer()
lb.fit(list(Class_dict.values()))
transformed_labels = lb.transform(names['str'])
y_bin_labels = []  

for i in range(transformed_labels.shape[1]):
    y_bin_labels.append('str' + str(i))
    names['str' + str(i)] = transformed_labels[:, i]

In [ ]:
Class_dict

In [ ]:
names.drop('str',axis=1,inplace=True)
names.drop('Types',axis=1,inplace=True)
names.head()

#### 1.3 Extraction & Input pipe 

In [ ]:
#analysis rectangular box value in xmls
def to_labels(path):
    xml = open('{}'.format(path)).read()                         #read xml in path 
    sel = etree.HTML(xml)                     
    width = int(sel.xpath('//size/width/text()')[0])     #extract the width/height
    height = int(sel.xpath('//size/height/text()')[0])    #extract the x,y value
    xmin = int(sel.xpath('//bndbox/xmin/text()')[0])
    xmax = int(sel.xpath('//bndbox/xmax/text()')[0])
    ymin = int(sel.xpath('//bndbox/ymin/text()')[0])
    ymax = int(sel.xpath('//bndbox/ymax/text()')[0])
    return [xmin/width, ymin/height, xmax/width, ymax/height]   #return the four relative points 

In [ ]:
#set value to labels
labels = [to_labels(path) for path in xmls_path]
labels[:3]

In [ ]:
#set four labels as outputs
out1,out2,out3,out4 = list(zip(*labels))        
#convert to np.array
out1 = np.array(out1)
out2 = np.array(out2)
out3 = np.array(out3)
out4 = np.array(out4)
label = np.array(names.values)

In [ ]:
#label to tf.data
label_datasets = tf.data.Dataset.from_tensor_slices((out1,out2,out3,out4,label))
label_datasets

In [ ]:
#def load_image function
def load_image(path):
    image = tf.io.read_file(path)                           
    image = tf.image.decode_jpeg(image,3)               
    image = tf.image.resize(image,[224,224])               
    image = tf.cast(image/127.5-1,tf.float32)                 
    return image      

In [ ]:
#build dataset
dataset = tf.data.Dataset.from_tensor_slices(imgs_train)
dataset = dataset.map(load_image)

In [ ]:
dataset_label = tf.data.Dataset.zip((dataset,label_datasets))

In [ ]:
#batch constant
BATCH_SIZE = 16
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
#batch extraction and shuffle
dataset_label = dataset_label.repeat().shuffle(500).batch(BATCH_SIZE)
dataset_label = dataset_label.prefetch(AUTO)

In [ ]:
#Split dataset
test_count = int(len(imgs_train)*0.2)
train_count = len(imgs_train) - test_count
test_count,train_count

In [ ]:
train_dataset = dataset_label.skip(test_count)
test_dataset = dataset_label.take(test_count)

In [ ]:
train_dataset

In [ ]:
species_dict = {v:k for k,v in Class_dict.items()}

In [ ]:
#check from train_data
for img, label in train_dataset.take(1):
    plt.imshow(keras.preprocessing.image.array_to_img(img[0]))     
    out1,out2,out3,out4,out5 = label                            
    xmin,ymin,xmax,ymax = out1[0].numpy()*224,out2[0].numpy()*224,out3[0].numpy()*224,out4[0].numpy()*224
    rect = Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),fill=False,color='r')  
    ax = plt.gca()                      
    ax.axes.add_patch(rect)   
    pred_imglist = []
    pred_imglist.append(species_dict[np.argmax(out5[0])+1])
    plt.title(pred_imglist)
    plt.show()

### 2. Model Building

#### 2.1 The VGG16 model 

In [ ]:
#Convolution based
conv = keras.applications.xception.Xception(weights='imagenet',
                                            include_top=False,
                                            input_shape=(224,224,3),
                                            pooling='avg')

In [ ]:
#open trainable
conv.trainable = True

In [ ]:
#define Conv + FC structure
inputs = keras.Input(shape=(224,224,3))
x = conv(inputs)
x1 = keras.layers.Dense(1024,activation='relu')(x)
x1 = keras.layers.Dense(512,activation='relu')(x1)


out1 = keras.layers.Dense(1,name='out1')(x1)
out2 = keras.layers.Dense(1,name='out2')(x1)
out3 = keras.layers.Dense(1,name='out3')(x1)
out4 = keras.layers.Dense(1,name='out4')(x1)

x2 = keras.layers.Dense(1024,activation='relu')(x)
x2 = keras.layers.Dropout(0.5)(x2)
x2 = keras.layers.Dense(512,activation='relu')(x2)
out_class = keras.layers.Dense(10,activation='softmax',name='out_item')(x2)

out = [out1,out2,out3,out4,out_class]

model = keras.models.Model(inputs=inputs,outputs=out)
model.summary()

#### 2.2 Compile & Fitting

In [ ]:
#model compille
model.compile(keras.optimizers.Adam(0.0003),
              loss={'out1':'mse',
                    'out2':'mse',
                    'out3':'mse',
                    'out4':'mse',
                    'out_item':'categorical_crossentropy'},
              metrics=['mae','acc'])

In [ ]:
#learning_rate reduce module
lr_reduce = keras.callbacks.ReduceLROnPlateau('val_loss', patience=6, factor=0.5, min_lr=1e-6)

In [ ]:
history = model.fit(train_dataset,
                   steps_per_epoch=train_count//BATCH_SIZE,
                   epochs=200,
                   validation_data=test_dataset,
                   validation_steps=test_count//BATCH_SIZE)

In [ ]:
#training visualization
def plot_history(history):                
    hist = pd.DataFrame(history.history)           
    hist['epoch']=history.epoch
    
    plt.figure()                                     
    plt.xlabel('Epoch')
    plt.ylabel('MSE')               
    plt.plot(hist['epoch'],hist['loss'],
            label='Train Loss')
    plt.plot(hist['epoch'],hist['val_loss'],
            label='Val Loss')                           
    plt.legend()
    
    plt.figure()                                      
    plt.xlabel('Epoch')
    plt.ylabel('Val_MAE')               
    plt.plot(hist['epoch'],hist['val_out1_mae'],
            label='Out1_mae')
    plt.plot(hist['epoch'],hist['val_out2_mae'],
            label='Out2_mae')
    plt.plot(hist['epoch'],hist['val_out3_mae'],
            label='Out3_mae')
    plt.plot(hist['epoch'],hist['val_out4_mae'],
            label='Out4_mae')
    plt.legend()      
    
    plt.figure()                                      
    plt.xlabel('Epoch')
    plt.ylabel('Val_Item_Acc')               
    plt.plot(hist['epoch'],hist['val_out_item_acc'],
            label='Out5_acc')
    
    plt.show()
    
plot_history(history)      

#### 2.3 Model Evaluation

In [ ]:
mae = model.evaluate(test_dataset)

In [ ]:
print('out1_mae in test:{}'.format(mae[6]))
print('out2_mae in test:{}'.format(mae[8]))
print('out3_mae in test:{}'.format(mae[10]))
print('out4_mae in test:{}'.format(mae[12]))
print('class_label in test:{}'.format(mae[15]))

In [ ]:
model.save("class_location.h5")

In [ ]:
species_dict = {v:k for k,v in Class_dict.items()}

In [ ]:
species_dict

In [ ]:
plt.figure(figsize=(10,24))
for img,_ in train_dataset.take(1):
    out1,out2,out3,out4,label = model.predict(img)
    for i in range(3):
        plt.subplot(3,1,i+1)            
        plt.imshow(keras.preprocessing.image.array_to_img(img[i]))    
        pred_imglist = []
        pred_imglist.append(species_dict[np.argmax(out5[i])+1])
        plt.title(pred_imglist)
        xmin,ymin,xmax,ymax = out1[i]*224,out2[i]*224,out3[i]*224,out4[i]*224
        rect = Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),fill=False,color='r') 
        ax = plt.gca()                   
        ax.axes.add_patch(rect)        